### **initializing important libraries & dataset**

In [1]:
#pandas for dataset
import pandas as pd

#modeling neural networks including CNNs
import torch
import torch.nn as nn

#invoke various optimizers
import torch.optim as optim
import torch.nn.functional as F

#loading and checking the dataset
from tensorflow import keras

#plot raw images
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from sklearn.preprocessing import OneHotEncoder

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

#convert all data to flot32, else PyTorch will cry
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#add a channel dimension (1 for grayscale)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print ("Rows and columns in training data inputs (i.e. images): ", x_train.shape)
print ("Y_actual shape: ", y_train.shape)

#make a list of lists
y_train = [[y] for y in y_train]

encoder = OneHotEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train).toarray()

print ("Printing the one-hot-encoded values for the class")
print(y_train)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Rows and columns in training data inputs (i.e. images):  (60000, 28, 28, 1)
Y_actual shape:  (60000,)
Printing the one-hot-encoded values for the class
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [3]:
#we make minibatches in a data loader and we specify the size of the minibatch here.
batch_size = 32

#convert the numpy arrays into torch tensors
x_train = torch.tensor(x_train)
y_train = torch.tensor(y_train)

# Torch expects the format to be (# Channels, Height, Width format)
x_train = x_train.permute(0, 3, 1, 2) #batch, channel, height, width reshaping

train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

## Exercise E1. Define the following CNN network and train on GPU
In a separate notebook, implement the following network and train->test it.

**Architecture:**

- **Input Layer**:
  - The network takes grayscale images as input, where each image has a single channel (1 for grayscale).
  
- **Convolutional Layer 1**:
  - Applies the first convolution operation using `nn.Conv2d(1, 32, kernel_size=5)`.
  - Uses 32 filters with a kernel size of 5x5.
  
- **ReLU Activation 1**:
  - Applies the Rectified Linear Unit (ReLU) activation function to introduce non-linearity after the first convolutional layer.
  
- **Max Pooling Layer 1**:
  - Performs max-pooling with a 2x2 kernel to downsample the feature maps obtained from the first convolutional layer.
  
- **Convolutional Layer 2**:
  - Applies the second convolution operation using `nn.Conv2d(32, 64, kernel_size=5)`.
  - Uses 64 filters with a kernel size of 5x5.
  
- **ReLU Activation 2**:
  - Applies the ReLU activation function after the second convolutional layer.
  
- **Max Pooling Layer 2**:
  - Performs max-pooling with a 2x2 kernel to downsample the feature maps obtained from the second convolutional layer.
  
- **Fully Connected Layer 1 (fc1)**:
  - Reshapes the 64x4x4 feature maps into a flattened vector.
  - Connects to a fully connected layer with 128 neurons.
  
- **ReLU Activation 3**:
  - Applies the ReLU activation function after the first fully connected layer.
  
- **Fully Connected Layer 2 (fc2)**:
  - Connects to a fully connected layer with 10 neurons, corresponding to the 10 possible classes in the MNIST dataset (digits 0-9).
  
- **Output Layer**:
  - The output layer provides the final classification results.
  - The network predicts class probabilities for each of the 10 classes using a softmax activation function.
  
- **Forward Pass**:
  - In the `forward` method, the input data `x` is passed through each layer sequentially.
  - Convolutions, ReLU activations, and max-pooling operations are applied in the specified order.
  - After max-pooling, the feature maps are flattened into a vector.
  - The flattened vector is passed through the fully connected layers with ReLU activations.
  - Finally, the network produces class scores as output.

  Once the network is trained, print the final training loss, then compute accuracy on the text data.



In [4]:
'''Architecture'''
class CNNNetwork(nn.Module):
    def __init__(self, num_classes=10):
        super(CNNNetwork, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv1_act = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_act = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 4 * 4, 128)
        self.act3 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool1(self.conv1_act(self.conv1(x)))
        x = self.pool2(self.conv2_act(self.conv2(x)))
        x = self.flatten(x)
        x = self.act3(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
#create an instance of the network
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNNetwork().to(device)  # Move the model to the appropriate device

#print the model architecture
print(model)

CNNNetwork(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv1_act): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv2_act): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (act3): ReLU()
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [6]:
#initialize the model, loss function, and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

#train the model for 10 epochs
num_epochs = 10
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)  # Ensure data is on the same device as the model
        optimizer.zero_grad()
        outputs = model(images)  # forward pass
        loss = criterion(outputs, labels)  # error / loss computation
        loss.backward()  # gradient Computation
        optimizer.step()  # backpropagation
    print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

#save the final model
torch.save(model.state_dict(), 'mnist_cnn_model.pth')


Epoch [1/10], Step [1875/1875], Loss: 0.0496
Epoch [2/10], Step [1875/1875], Loss: 0.1055
Epoch [3/10], Step [1875/1875], Loss: 0.0088
Epoch [4/10], Step [1875/1875], Loss: 0.0135
Epoch [5/10], Step [1875/1875], Loss: 0.0002
Epoch [6/10], Step [1875/1875], Loss: 0.0023
Epoch [7/10], Step [1875/1875], Loss: 0.0084
Epoch [8/10], Step [1875/1875], Loss: 0.0066
Epoch [9/10], Step [1875/1875], Loss: 0.0002
Epoch [10/10], Step [1875/1875], Loss: 0.0004


In [7]:
total = 0
correct = 0
for i, image in enumerate(x_test):
    image = image.reshape(1, 1, 28, 28)  # batch_size, channel, height, width
    image = torch.Tensor(image).to(device)  # Ensure test data is on the same device
    output = model(image)[0]  # since batch size is 1, we will get only one output
    y_pred = torch.argmax(output).item()
    y_actual = y_test[i]
    if y_actual == y_pred:
        correct += 1
    total += 1
print (f"Accuracy = {correct / total * 100:.2f} %")

Accuracy = 98.58 %


The CNN model trained on the MNIST dataset achieved 98.58% accuracy on the test set, indicating strong performance.
The training loss decreased steadily across 10 epochs, with the final loss values reaching near-zero, suggesting efficient learning.
Observations:

The model architecture is well-structured, with two convolutional layers, max-pooling, and fully connected layers, allowing for hierarchical feature extraction.
The use of ReLU activation in convolutional and fully connected layers ensures non-linearity, improving model expressiveness.
The Adam optimizer was chosen for training, which is a good default optimizer due to its adaptive learning rate capability.

The final loss values are extremely low (e.g., 0.0002 at epoch 9), which may indicate overfitting to the training data.
The lack of dropout or weight regularization suggests the model could have poor generalization to slightly altered images.

## Exercise E2. Design a modified version of E1's model.

1. Introduce a dropout layer (between with a specified dropout probability (e.g., `dropout_prob=0.2`) between  the Fully Connected Layer 1 and Fully Connected Layer 2. Dropout helps prevent overfitting by randomly setting a fraction of the input units to zero during each forward pass.

**Hint:** inside __init__(), define a dropout layer using `self.dropout = nn.Dropout(p=0.5)` and call it within the `forward()` function after calling a layer to apply dropout to that layer.

2. Create an instance of the modified model with dropout. Copy the training pipeline code and testing code  and repeat the training and testing process. Do you see any improvement in test accuracy after training with dropout?

3. Apply L1 and L2 regularizations following the advise here: https://stackoverflow.com/questions/44641976/pytorch-how-to-add-l1-regularizer-to-activations

4. Play with  hyper-parameters such as learning rate, dropout probabilities and also feel free increase the number of epochs. Are the results improving? Write down your insights in a markdown block.

In [8]:
#modified CNN with Dropout
class ModifiedCNN(nn.Module):
    def __init__(self, dropout_prob=0.5):
        super(ModifiedCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2)

        self.fc1 = nn.Linear(64 * 4 * 4, 128)
        self.dropout = nn.Dropout(p=dropout_prob)  #apply Dropout
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  # dropout applied
        out = self.fc2(x)

        return out


# reate model instance
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ModifiedCNN(dropout_prob=0.5).to(device)


In [9]:
#initialize the model, loss function, and optimizer
criterion = nn.CrossEntropyLoss()
lambda1, lambda2 = 0.001, 0.001  # L1 & L2 penalty factors
optimizer = optim.Adam(model.parameters(), lr=0.001)

#fefine loss threshold
loss_threshold = 0.38
epoch = 0
running_loss = float('inf')

#train until loss goes below the threshold
while running_loss > loss_threshold:
    model.train()
    running_loss = 0.0
    epoch += 1  #track number of epochs

    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs, _ = model(images)  #forward pass
        loss = criterion(outputs, labels)  #compute cross-entropy loss

        #compute L1 & L2 Regularization
        l1_regularization = lambda1 * sum(torch.norm(p, 1) for p in model.parameters())
        l2_regularization = lambda2 * sum(torch.norm(p, 2) for p in model.parameters())

        #total loss = Cross Entropy + L1 + L2 (stackoverflow)
        loss += l1_regularization + l2_regularization

        loss.backward()  #gradient
        optimizer.step()  #backprop

        running_loss += loss.item()

    running_loss /= len(train_loader)  #average loss per epoch
    print(f'Epoch [{epoch}], Loss: {running_loss:.4f}')

#save the final model
torch.save(model.state_dict(), 'mnist_cnn_dropout_model.pth')
print(f"Training completed in {epoch} epochs!")


Epoch [1], Loss: 1.2139
Epoch [2], Loss: 0.4912
Epoch [3], Loss: 0.4521
Epoch [4], Loss: 0.4486
Epoch [5], Loss: 0.4459
Epoch [6], Loss: 0.4379
Epoch [7], Loss: 0.4353
Epoch [8], Loss: 0.4278
Epoch [9], Loss: 0.4260
Epoch [10], Loss: 0.4199
Epoch [11], Loss: 0.4171
Epoch [12], Loss: 0.4155
Epoch [13], Loss: 0.4062
Epoch [14], Loss: 0.4099
Epoch [15], Loss: 0.4113
Epoch [16], Loss: 0.4089
Epoch [17], Loss: 0.4005
Epoch [18], Loss: 0.4108
Epoch [19], Loss: 0.4000
Epoch [20], Loss: 0.4073
Epoch [21], Loss: 0.4008
Epoch [22], Loss: 0.4029
Epoch [23], Loss: 0.3995
Epoch [24], Loss: 0.4011
Epoch [25], Loss: 0.3967
Epoch [26], Loss: 0.3931
Epoch [27], Loss: 0.3998
Epoch [28], Loss: 0.3937
Epoch [29], Loss: 0.3964
Epoch [30], Loss: 0.4103
Epoch [31], Loss: 0.3969
Epoch [32], Loss: 0.4016
Epoch [33], Loss: 0.4005
Epoch [34], Loss: 0.3934
Epoch [35], Loss: 0.3895
Epoch [36], Loss: 0.4057
Epoch [37], Loss: 0.3965
Epoch [38], Loss: 0.3900
Epoch [39], Loss: 0.3967
Epoch [40], Loss: 0.3876
Epoch [41

i made the threshold 0.38 because i initially put it to 0.005 then 0.1, and i realized it'll never get there as epoch 80 still left me at around 0.38. I started out with 0.5 l1 penalty factor & 0.1 l2 penalty factor and that made the loss start at around 13, so i reduced it to 0.001. it helped a bit with the loss, but still not that good.

In [10]:
model.load_state_dict(torch.load('mnist_cnn_dropout_model.pth'))
model.eval()

total = 0
correct = 0

for i, image in enumerate(x_test):
    image = image.reshape(1, 1, 28, 28)  # batch_size, channel, height, width
    image = torch.Tensor(image).to(device)

    output, _ = model(image)
    y_pred = torch.argmax(output[0]).item()
    y_actual = y_test[i]

    if y_actual == y_pred:
        correct += 1
    total += 1

#print final accuracy
print(f"Accuracy = {correct / total * 100:.2f} %")


<ipython-input-10-f1398d00109a>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('mnist_cnn_dropout_model.pth'))


Accuracy = 98.50 %


Dropout layer (p=0.5) was used between fc1 and fc2 to reduce overfitting by randomly deactivating neurons during training.
L1 & L2 regularization was applied to the model parameters to penalize large weights and enforce sparsity.
Adaptive threshold-based training was used to continue training until loss fell below 0.38.

The initial loss was significantly higher, especially with stronger L1 and L2 regularization settings, which had to be reduced for better convergence.

Dropout prevented the model from memorizing the training set too strictly, leading to improved robustness.
L1 and L2 regularization helped avoid extremely large weights, making the model more stable but requiring careful tuning of penalty factors.
Hyperparameter adjustments (learning rate, dropout probability, regularization strength) were necessary to balance accuracy and training stability.

## **Key Takeaways**
The original CNN model (E1) was highly accurate but likely overfitted to the training data due to lack of regularization.
The modified CNN (E2) introduced dropout and weight penalties, making the model more generalizable but requiring longer training.
Despite a slight drop in test accuracy, E2's model is more robust to noise and unseen data due to dropout and regularization.
I think the trade-off between faster convergence (E1) vs. generalization (E2) highlights the importance of regularization techniques in deep learning.